In [ ]:
import os
import json
import re
from groq import Groq

client = Groq(
    api_key="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",
)

# Modify the prompt to request JSON output
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": (
                "Explain the importance of fast language models. "
                "Provide the answer in JSON format with keys 'summary', 'benefits', and 'applications'."
            ),
        }
    ],
    model="llama3-8b-8192",
)

content = chat_completion.choices[0].message.content

# Use regex to extract JSON content in case there is extra text
json_match = re.search(r'\{.*\}', content, re.DOTALL)

if json_match:
    json_content = json_match.group()
    try:
        data = json.loads(json_content)
        print(json.dumps(data, indent=4))
    except json.JSONDecodeError as e:
        print("Failed to parse JSON:", e)
        print("Content was:", json_content)
else:
    print("No JSON content found in the response.")


In [12]:
#Generate Keys

import requests

# Replace with your actual endpoint URL
url = "http://localhost:8000/generate_api_key"

# The master key from any service (ensure it's valid)
master_key = "your_master_service_api_key"  # Replace with the actual master key

# Prepare the request data
data = {
    "name": "Jane Doe",
    "email": "jane.doe@example.com",
    "privilege": "user",  # or "admin"
    "master_key": master_key
}

# Prepare headers
headers = {
    "Content-Type": "application/json"
}

# Send the POST request
response = requests.post(url, headers=headers, json=data)

# Handle the response
if response.status_code == 200:
    # Successful key generation
    result = response.json()
    print(f"Your new API key is: {result['api_key']}")
elif response.status_code == 400:
    # Bad request (e.g., invalid privilege level)
    print(f"Error {response.status_code}: {response.json()['detail']}")
elif response.status_code == 401:
    # Unauthorized (invalid master key)
    print(f"Error {response.status_code}: {response.json()['detail']}")
else:
    # Other errors
    print(f"Error {response.status_code}: {response.text}")

In [ ]:
#LLM Inference Default

import requests

api_key = "ljoWc7l91mdURBRNTNPlZQlGpV5OMo"  # Replace with your actual API key

url = "http://localhost:8000/chat/completions"
headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

data = {
    "messages": [
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
    "model": "llama3.1-8b-8192",
    "service_name": "cerebras",
}

response = requests.post(url, json=data, headers=headers)

if response.status_code == 200:
    result = response.json()
    print(result["content"])
else:
    print(f"Error {response.status_code}: {response.text}")

In [ ]:
#LLM Inference Custom Parameters

import requests

api_key = "ljoWc7l91mdURBRNTNPlZQlGpV5OMo"  # Replace with your actual API key

url = "http://localhost:8000/chat/completions"
headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

data = {
  "messages": [
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
  "model": "llama3.1-8b",
  "service_name": "cerebras",
  "temperature": 0.7,
  "max_tokens": 500,
  "top_p": 0.9,
  "stop": ["\n"]
}

response = requests.post(url, json=data, headers=headers)

if response.status_code == 200:
    result = response.json()
    print(result["content"])
else:
    print(f"Error {response.status_code}: {response.text}")

In [ ]:
#Delete Key

import requests

# Replace with your actual endpoint URL
url = "http://localhost:8000/delete_api_key"

# The admin's email and API key
admin_email = "admin@example.com"
admin_api_key = "admin_api_key_value"

# The email of the user whose API key you want to delete
email_to_delete = "user@example.com"

# Prepare the request data
data = {
    "admin_email": admin_email,
    "admin_api_key": admin_api_key,
    "email_to_delete": email_to_delete
}

# Prepare headers
headers = {
    "Content-Type": "application/json"
}

# Send the DELETE request
response = requests.delete(url, headers=headers, json=data)

# Handle the response
if response.status_code == 200:
    # Successful deletion
    print(response.json())
elif response.status_code == 404:
    # Email to delete not found
    print(f"Error {response.status_code}: {response.json()['detail']}")
elif response.status_code == 401:
    # Invalid admin API key
    print(f"Error {response.status_code}: {response.json()['detail']}")
elif response.status_code == 403:
    # Admin privilege required
    print(f"Error {response.status_code}: {response.json()['detail']}")
else:
    # Other errors
    print(f"Error {response.status_code}: {response.text}")


In [14]:
#Change privilege

import requests

# Replace with your actual endpoint URL
url = "http://localhost:8000/change_privilege"

# The admin's email and API key
admin_email = "admin@example.com"
admin_api_key = "admin_api_key_value"

# The email of the user whose privilege level you want to change
email_to_change = "user@example.com"

# The new privilege level ("user" or "admin")
new_privilege = "admin"

# Prepare the request data
data = {
    "admin_email": admin_email,
    "admin_api_key": admin_api_key,
    "email_to_change": email_to_change,
    "new_privilege": new_privilege
}

# Prepare headers
headers = {
    "Content-Type": "application/json"
}

# Send the PUT request
response = requests.put(url, headers=headers, json=data)

# Handle the response
if response.status_code == 200:
    # Successful privilege change
    print(response.json())
elif response.status_code == 404:
    # Email to change not found
    print(f"Error {response.status_code}: {response.json()['detail']}")
elif response.status_code == 401:
    # Invalid admin API key
    print(f"Error {response.status_code}: {response.json()['detail']}")
elif response.status_code == 403:
    # Admin privilege required
    print(f"Error {response.status_code}: {response.json()['detail']}")
elif response.status_code == 400:
    # Invalid privilege level
    print(f"Error {response.status_code}: {response.json()['detail']}")
else:
    # Other errors
    print(f"Error {response.status_code}: {response.text}")

In [ ]:
import requests

api_key = "ljoWc7l91mdURBRNTNPlZQlGpV5OMo"  # Replace with your actual API key
url = "http://localhost:8000/chat/completions"  # Adjust the URL if necessary

# System prompt
system_prompt = """
You're evaluating writing style in text.
Your evaluations must always be in JSON format. Here is an example JSON response:

{ "readability": 4, "conciseness": 2 }

"""

# User prompt
tweet = "This is a sample text that needs evaluation."
prompt = f"""
Evaluate the text:

{tweet}

You're evaluating the readability and conciseness with values from 0 (extremely bad) to 10 (extremely good).
"""

# Messages
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt}
]

# Request data
data = {
    "messages": messages,
    "model": "llama3.1-8b",  # Replace with your desired model
    "service_name": "cerebras",     # Replace with your desired service
    "temperature": 0.0,         # Low temperature for deterministic output
    "max_tokens": 100,          # Adjust as needed
    "top_p": 1.0,
}

headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

# Make the API call
response = requests.post(url, json=data, headers=headers)

if response.status_code == 200:
    result = response.json()
    print(result["content"])
else:
    print(f"Error {response.status_code}: {response.text}")

In [ ]:

model = ChatOllama(model="mistral:7b-instruct-v0.2-q4_1")
parser = StrOutputParser()
output = 'News_data_Company_SP500_3_Sentiment_Score.xlsx'
Question = """
    In this format respond to the news aricle given in the context interpret it and classify and respond in this JSON format only. No other things and extra explaination or interpretation.
    JSON = {
        Sentiment: Positive/Negative/Neutral,
        Corruption: Yes/No/Possible,
        Fraud: Yes/No/Possible,
        Type: (Type of news article),
        Explanation: (Write Explanation Here),
    }
    Return just the JSON No matter what. Strictly the formatted JSON.
"""

In [ ]:
def is_valid_json(json_data):
    try:
        json_object = json.loads(json_data)
    except ValueError as e:
        return False
    return True

def fix_json(json_data):
    try:
        if not json_data.strip().endswith('}'):
            json_data += '}'
        return json_data
    except Exception as e:
        return json_data
    
def perform_sentiment_analysis(text, model, parser, Question):
    template = """
    This is a news article from Thompson Reuters. Use this news article and respond only in the format asked in the question nothing extra. Do not respond with anything else just the JSON format.

    Context: {context}

    Question: {question}
    """

    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model | parser
    response = chain.invoke({
        "context": text,
        "question": Question
    })
    if not is_valid_json(response):
        fixed_response = fix_json(response)
        if is_valid_json(fixed_response):
            return json.loads(fixed_response)
        else:
            response = {
                'Sentiment': None,
                'Corruption': None,
                'Fraud': None,
                'Type': None,
                'Explanation': "Error in Analysis"
            }
            return(response)
    else:
        return json.loads(response)